In [55]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
#For other editors just put the pathings to a holder folder

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

file_path = '/kaggle/input/ethereum-effect-pc-parts/FACT_RAM_PRICE.csv'
file_path2 = '/kaggle/input/edited-dim-ram-prod/DIM_RAM_PROD-1.csv'
df = pd.read_csv(file_path)
df1 = pd.read_csv(file_path2)

# merge the dataframes by ID
merged_df = df.merge(df1, left_on='ProdId', right_on='Id')

for x in range(100):
    if x % 2 == 1 or x % 5 == 0:
        merged_df.loc[x, 'Price_USD'] = np.NaN
    if x % 2 == 0 and x % 3 == 0:
        merged_df.loc[x, 'Speed'] = np.NaN

# shortens dataframe to make it manageable
merged_df = merged_df.head(100)

# interpolate the data
merged_df = merged_df.interpolate()

# interpolating doesn't remove the first NaN values, so we use bfill to get rid of those
merged_df = merged_df.bfill()

# now we filter by pricing
merged_df = merged_df[merged_df['Price_USD'] < 13]
merged_df = merged_df.reset_index()

# an index column is added from when we reset the index, so we drop this column
merged_df = merged_df.drop('index', axis=1)
merged_df

# more random ways of inserting NA values
for x in range(len(merged_df)):
    if x % 2 == 1:
        merged_df.loc[x, 'Speed'] = np.NaN
    if x % 3 == 0:
        merged_df.loc[x, 'Price_Original'] = np.NaN
        
        
merged_df = merged_df.fillna(0)
merged_df = merged_df.sort_values(by='Price_Original')

# now we have an extremely small dataframe with a particular set of gpus (only 12 left)
# take out the rows that were filled with 0s using filtering
merged_df = merged_df[(merged_df['Price_Original'] != 0) & (merged_df['Speed'] != 0)]

# 4 GPUS priced less than 13 USD whos original index was not used to insert NA values
merged_df

,ProdId,TimeId,RegionId,MerchantId,Price_USD,Price_Original,Id,Manufacturer,RAM_Name,Memory_Type,Speed,Capacity
2,1,20130622,4,32,12.936925,9.795,1,ADATA,Adata,DDR,400.0,0.5
4,1,20130624,4,32,12.853993,9.795,1,ADATA,Adata,DDR,400.0,0.5
8,1,20130628,4,32,12.868519,9.870,1,ADATA,Adata,DDR,400.0,0.5
10,1,20130630,4,32,12.860377,9.885,1,ADATA,Adata,DDR,400.0,0.5
